In [39]:
import pandas as pd
import geopandas as gpd
import shapely
import numpy as np
from sqlalchemy import create_engine
from MyCreds.mycreds import Capstone_AWS_PG
from geoalchemy2 import Geometry

In [17]:
parks = gpd.read_file('../data/parks/parks-geojson.geojson')
parks

,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.96086 1.34618 0.00000)
1,kml_2,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.97898 1.38990 0.00000)
2,kml_3,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.92313 1.40991 0.00000)
3,kml_4,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.86697 1.37924 0.00000)
4,kml_5,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.82907 1.37276 0.00000)
...,...,...,...
345,kml_346,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.89036 1.36796 0.00000)
346,kml_347,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.98919 1.39017 0.00000)
347,kml_348,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.88595 1.35520 0.00000)
348,kml_349,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.90374 1.39164 0.00000)


In [18]:
parks = parks.to_crs(4326)
parks['geometry'] = parks['geometry'].map(lambda point: shapely.ops.transform(lambda x, y, z: (y, x), point))
parks

,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,POINT (1.34618 103.96086)
1,kml_2,<center><table><tr><th colspan='2' align='cent...,POINT (1.38990 103.97898)
2,kml_3,<center><table><tr><th colspan='2' align='cent...,POINT (1.40991 103.92313)
3,kml_4,<center><table><tr><th colspan='2' align='cent...,POINT (1.37924 103.86697)
4,kml_5,<center><table><tr><th colspan='2' align='cent...,POINT (1.37276 103.82907)
...,...,...,...
345,kml_346,<center><table><tr><th colspan='2' align='cent...,POINT (1.36796 103.89036)
346,kml_347,<center><table><tr><th colspan='2' align='cent...,POINT (1.39017 103.98919)
347,kml_348,<center><table><tr><th colspan='2' align='cent...,POINT (1.35520 103.88595)
348,kml_349,<center><table><tr><th colspan='2' align='cent...,POINT (1.39164 103.90374)


In [20]:
from bs4 import BeautifulSoup
import re
from collections import defaultdict

In [21]:
des = parks['Description']

In [22]:
parks_dict = defaultdict(dict)

for i, description in enumerate(des):
    soup = BeautifulSoup(des[i])
    table = soup.find_all('tr')[1:]

    parks_dict[i] = {re.findall('<th>(\w+)<\/th>\s<td>(.+)?<\/td>', str(row))[0][0]:
                           re.findall('<th>(\w+)<\/th>\s<td>(.+)?<\/td>', str(row))[0][1] for row in table}

In [23]:
parks_descriptions = pd.DataFrame(parks_dict).T
parks_descriptions

,ADDRESSBLOCKHOUSENUMBER,ADDRESSBUILDINGNAME,ADDRESSTYPE,HYPERLINK,LANDXADDRESSPOINT,LANDYADDRESSPOINT,NAME,PHOTOURL,ADDRESSPOSTALCODE,DESCRIPTION,ADDRESSSTREETNAME,ADDRESSFLOORNUMBER,INC_CRC,FMEL_UPD_D,ADDRESSUNITNUMBER
0,,,,,42193.6328,36479.08,Jalan Pelatok Open Space,,,Jalan Pelatok,,,BC5EA394BD8CE9F8,20200218182414,
1,,,,,44210.12,41314.54,Turnhouse Park,,,Natheravon Road,,,AB31CCDD36F93C1C,20200218182414,
2,,,,www.nparks.gov.sg/gardens-parks-and-nature/par...,37994.87,43526.7266,Coney Island Park,,,Beside Punggol Promenade Nature Walk,,,7CAC664DFF94240C,20200218182414,
3,,,,,31744.6465,40134.5625,Stratton Walk Playground,,,Located at the end of Stratton Walk,,,8FC92F17CF75A88A,20200218182414,
4,,,,,27527.52,39418.4531,Leban Park,,,Behind Sembawang Hills Food Centre. Upper Tho...,,,EB7459375A3D10F7,20200218182414,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,,,,,34348.5273,38888.0625,Realty Park,,,Along Hougang Ave 4,,,A7BB1BF496807A8B,20200218182414,
346,,,,,45346.4766,41344.2461,Changi Point Promenade,,,Near Lor Bekukong,,,5FEA22BCC48153DA,20200218182414,
347,,,,,33857.832,37476.270000000004,Aroozoo Avenue Playground,,,Bounded by Aroozoo Ave and Jalan Mastuli,,,ACBC9E152D19BD06,20200218182414,
348,,,,,35836.5352,41506.1133,St. Anne's Wood Playground,,,Inside the private housing estate at St Anne's...,,,43A081292A7AE024,20200218182414,


In [26]:
parks_descriptions.replace(r'^\s*$', np.nan, regex=True, inplace=True)
parks_descriptions.dropna(axis=1, inplace=True)
parks_descriptions

,LANDXADDRESSPOINT,LANDYADDRESSPOINT,NAME,INC_CRC,FMEL_UPD_D
0,42193.6328,36479.08,Jalan Pelatok Open Space,BC5EA394BD8CE9F8,20200218182414
1,44210.12,41314.54,Turnhouse Park,AB31CCDD36F93C1C,20200218182414
2,37994.87,43526.7266,Coney Island Park,7CAC664DFF94240C,20200218182414
3,31744.6465,40134.5625,Stratton Walk Playground,8FC92F17CF75A88A,20200218182414
4,27527.52,39418.4531,Leban Park,EB7459375A3D10F7,20200218182414
...,...,...,...,...,...
345,34348.5273,38888.0625,Realty Park,A7BB1BF496807A8B,20200218182414
346,45346.4766,41344.2461,Changi Point Promenade,5FEA22BCC48153DA,20200218182414
347,33857.832,37476.270000000004,Aroozoo Avenue Playground,ACBC9E152D19BD06,20200218182414
348,35836.5352,41506.1133,St. Anne's Wood Playground,43A081292A7AE024,20200218182414


In [29]:
parks.rename({'Name': 'ID'}, axis=1, inplace=True)
parks_full = parks.merge(parks_descriptions, left_index=True, right_index=True)
parks_full

,ID,Description,geometry,LANDXADDRESSPOINT,LANDYADDRESSPOINT,NAME,INC_CRC,FMEL_UPD_D
0,kml_1,<center><table><tr><th colspan='2' align='cent...,POINT (1.34618 103.96086),42193.6328,36479.08,Jalan Pelatok Open Space,BC5EA394BD8CE9F8,20200218182414
1,kml_2,<center><table><tr><th colspan='2' align='cent...,POINT (1.38990 103.97898),44210.12,41314.54,Turnhouse Park,AB31CCDD36F93C1C,20200218182414
2,kml_3,<center><table><tr><th colspan='2' align='cent...,POINT (1.40991 103.92313),37994.87,43526.7266,Coney Island Park,7CAC664DFF94240C,20200218182414
3,kml_4,<center><table><tr><th colspan='2' align='cent...,POINT (1.37924 103.86697),31744.6465,40134.5625,Stratton Walk Playground,8FC92F17CF75A88A,20200218182414
4,kml_5,<center><table><tr><th colspan='2' align='cent...,POINT (1.37276 103.82907),27527.52,39418.4531,Leban Park,EB7459375A3D10F7,20200218182414
...,...,...,...,...,...,...,...,...
345,kml_346,<center><table><tr><th colspan='2' align='cent...,POINT (1.36796 103.89036),34348.5273,38888.0625,Realty Park,A7BB1BF496807A8B,20200218182414
346,kml_347,<center><table><tr><th colspan='2' align='cent...,POINT (1.39017 103.98919),45346.4766,41344.2461,Changi Point Promenade,5FEA22BCC48153DA,20200218182414
347,kml_348,<center><table><tr><th colspan='2' align='cent...,POINT (1.35520 103.88595),33857.832,37476.270000000004,Aroozoo Avenue Playground,ACBC9E152D19BD06,20200218182414
348,kml_349,<center><table><tr><th colspan='2' align='cent...,POINT (1.39164 103.90374),35836.5352,41506.1133,St. Anne's Wood Playground,43A081292A7AE024,20200218182414


In [32]:
parks_full = parks_full[['ID', 'LANDXADDRESSPOINT',
                         'LANDYADDRESSPOINT', 'NAME', 'INC_CRC', 'FMEL_UPD_D', 'geometry']]
parks_full.columns = [col.lower() for col in parks_full.columns]
parks_full

,id,landxaddresspoint,landyaddresspoint,name,inc_crc,fmel_upd_d,geometry
0,kml_1,42193.6328,36479.08,Jalan Pelatok Open Space,BC5EA394BD8CE9F8,20200218182414,POINT (1.34618 103.96086)
1,kml_2,44210.12,41314.54,Turnhouse Park,AB31CCDD36F93C1C,20200218182414,POINT (1.38990 103.97898)
2,kml_3,37994.87,43526.7266,Coney Island Park,7CAC664DFF94240C,20200218182414,POINT (1.40991 103.92313)
3,kml_4,31744.6465,40134.5625,Stratton Walk Playground,8FC92F17CF75A88A,20200218182414,POINT (1.37924 103.86697)
4,kml_5,27527.52,39418.4531,Leban Park,EB7459375A3D10F7,20200218182414,POINT (1.37276 103.82907)
...,...,...,...,...,...,...,...
345,kml_346,34348.5273,38888.0625,Realty Park,A7BB1BF496807A8B,20200218182414,POINT (1.36796 103.89036)
346,kml_347,45346.4766,41344.2461,Changi Point Promenade,5FEA22BCC48153DA,20200218182414,POINT (1.39017 103.98919)
347,kml_348,33857.832,37476.270000000004,Aroozoo Avenue Playground,ACBC9E152D19BD06,20200218182414,POINT (1.35520 103.88595)
348,kml_349,35836.5352,41506.1133,St. Anne's Wood Playground,43A081292A7AE024,20200218182414,POINT (1.39164 103.90374)


In [41]:
engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

sql_alc_cnxn = engine.connect()

# parks_full.to_postgis('parks_points', sql_alc_cnxn, if_exists='replace', index=False)

parks_full.to_postgis('parks_points',engine, if_exists='replace', index=False, dtype={'geometry': Geometry(geometry_type='POINT', srid= 4326)})

sql_alc_cnxn.close()

# Parks and Open Space

In [36]:
pos = gpd.read_file('../data/parks-and-open-space/mp14-sdcp-pw-plan-parks-and-open-space-shp/G_MP14_PKWB_PARKS_PL.shp')
pos

,OBJECTID,OID_1,CLASSIFCTN,PRP_STATUS,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,SHAPE_Leng,SHAPE_Area,geometry
0,1,0,NEIGHBOURHOOD PARK,EXISTING,592C7550177BF268,2016-07-18,27919.5872,45166.4263,283.063527,3193.830838,"POLYGON ((27976.462 45177.981, 27975.009 45149..."
1,2,0,NEIGHBOURHOOD PARK,EXISTING,E9A1AC47C9DC8698,2016-07-18,23205.8335,46915.3998,887.637083,19909.684922,"POLYGON ((23058.397 46880.819, 23058.405 46880..."
2,3,0,NEIGHBOURHOOD PARK,EXISTING,D7DF045D6805B00D,2016-07-18,24735.6118,47398.1171,503.291711,14221.679594,"POLYGON ((24754.238 47345.751, 24748.268 47342..."
3,4,0,NEIGHBOURHOOD PARK,EXISTING,F9D174E1E0E0111C,2016-07-18,24305.6678,46590.1864,942.984201,16383.964451,"POLYGON ((24481.037 46693.444, 24453.464 46681..."
4,5,0,NEIGHBOURHOOD PARK,EXISTING,C399D41D7CB9B4A5,2016-07-18,15074.8532,35773.5449,376.819317,1840.105171,"POLYGON ((15134.200 35814.452, 15127.450 35810..."
...,...,...,...,...,...,...,...,...,...,...,...
1102,1103,0,MARINE PARK,PLANNED,3A8CBA5EE0DFA259,2016-07-18,29449.7895,22378.3747,3421.757359,107711.294857,"POLYGON ((29653.515 22183.275, 29652.952 22173..."
1103,1104,0,NEIGHBOURHOOD PARK,PLANNED,5F7F93191F2E4D83,2016-07-18,24580.0774,29060.4320,339.151069,6169.308722,"POLYGON ((24581.417 29132.692, 24606.464 29077..."
1104,1105,0,REGIONAL PARK,EXISTING,740FE78FC19EC02A,2016-07-18,26065.4119,32926.0370,4962.018398,529242.527514,"POLYGON ((26014.034 33875.224, 26068.901 33869..."
1105,1106,0,REGIONAL PARK,PLANNED,3599F642FE503E51,2016-07-18,25464.4604,32890.8773,1522.745875,79886.842508,"POLYGON ((25393.986 33101.766, 25410.965 33087..."


In [37]:
pos = pos.to_crs(4326)
pos

,OBJECTID,OID_1,CLASSIFCTN,PRP_STATUS,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,SHAPE_Leng,SHAPE_Area,geometry
0,1,0,NEIGHBOURHOOD PARK,EXISTING,592C7550177BF268,2016-07-18,27919.5872,45166.4263,283.063527,3193.830838,"POLYGON ((103.83311 1.42485, 103.83309 1.42459..."
1,2,0,NEIGHBOURHOOD PARK,EXISTING,E9A1AC47C9DC8698,2016-07-18,23205.8335,46915.3998,887.637083,19909.684922,"POLYGON ((103.78891 1.44025, 103.78891 1.44025..."
2,3,0,NEIGHBOURHOOD PARK,EXISTING,D7DF045D6805B00D,2016-07-18,24735.6118,47398.1171,503.291711,14221.679594,"POLYGON ((103.80415 1.44445, 103.80410 1.44443..."
3,4,0,NEIGHBOURHOOD PARK,EXISTING,F9D174E1E0E0111C,2016-07-18,24305.6678,46590.1864,942.984201,16383.964451,"POLYGON ((103.80170 1.43855, 103.80145 1.43844..."
4,5,0,NEIGHBOURHOOD PARK,EXISTING,C399D41D7CB9B4A5,2016-07-18,15074.8532,35773.5449,376.819317,1840.105171,"POLYGON ((103.71771 1.34016, 103.71765 1.34013..."
...,...,...,...,...,...,...,...,...,...,...,...
1102,1103,0,MARINE PARK,PLANNED,3A8CBA5EE0DFA259,2016-07-18,29449.7895,22378.3747,3421.757359,107711.294857,"POLYGON ((103.84818 1.21689, 103.84817 1.21680..."
1103,1104,0,NEIGHBOURHOOD PARK,PLANNED,5F7F93191F2E4D83,2016-07-18,24580.0774,29060.4320,339.151069,6169.308722,"POLYGON ((103.80260 1.27974, 103.80283 1.27924..."
1104,1105,0,REGIONAL PARK,EXISTING,740FE78FC19EC02A,2016-07-18,26065.4119,32926.0370,4962.018398,529242.527514,"POLYGON ((103.81547 1.32263, 103.81597 1.32258..."
1105,1106,0,REGIONAL PARK,PLANNED,3599F642FE503E51,2016-07-18,25464.4604,32890.8773,1522.745875,79886.842508,"POLYGON ((103.80990 1.31564, 103.81005 1.31550..."


In [38]:
pos.dtypes

OBJECTID         int64
OID_1            int64
CLASSIFCTN      object
PRP_STATUS      object
INC_CRC         object
FMEL_UPD_D      object
X_ADDR         float64
Y_ADDR         float64
SHAPE_Leng     float64
SHAPE_Area     float64
geometry      geometry
dtype: object

In [40]:
pos.to_postgis('parks_open_space_poly',engine, if_exists='replace', index=False, dtype={'geometry': Geometry(geometry_type='POLYGON', srid= 4326)})